In [77]:
from ultralytics import YOLO
import torch
import cpuinfo
import GPUtil
import psutil
import numpy as np
import pandas as pd
import random
import os
import time
import csv
import gc

#### Функция получения параметров весов и смещений модели

In [78]:
def count_parameters(model):
    model = model.model
    param_dict = dict()
    total_params = 0
    for name, parameter in model.named_parameters():
        params = parameter.numel()
        param_dict[name] = params
        total_params += params
    return param_dict, total_params

#### Функция для получения случайных изображений из директории

In [79]:
folder_path = 'C:\\Users\\Artem\\datasets\\yolo\\person\\train\\images'

In [80]:
def get_random_images_from_folder(folder_path, num_images):
    # Получаем список всех файлов в папке
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    # Проверяем, достаточно ли файлов для выборки
    if num_images > len(all_files):
        raise ValueError(f"Запрошено {num_images} изображений, но в папке только {len(all_files)} доступных файлов.")
    
    # Выбираем случайные изображения
    selected_files = random.sample(all_files, num_images)
    return selected_files

#### Варьируемые параметры

In [81]:
# константы
ram = int(np.round(psutil.virtual_memory().total / (1024. **3)))
cpu = cpuinfo.get_cpu_info()['brand_raw']
gpu = GPUtil.getGPUs()[0].name
gpu_mem = GPUtil.getGPUs()[0].memoryTotal / 1024

used_gpu = False # в train device=cuda

# model_names7 yolov7 not supported in ultralytics
model_names5 = ['yolov5nu.pt', 'yolov5su.pt', 'yolov5mu.pt', 'yolov5lu.pt']
model_names6 = ['yolov6n.yaml', 'yolov6s.yaml', 'yolov6m.yaml', 'yolov6l.yaml']
model_names8 = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt']
model_names9 = ['yolov9t.pt', 'yolov9s.pt', 'yolov9m.pt', 'yolov9c.pt']
model_names10 = ['yolov10n.pt', 'yolov10s.pt', 'yolov10m.pt', 'yolov10l.pt']
model_names11 = ['yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt', 'yolo11l.pt']

models = [model_names5, model_names6, model_names8, model_names9, model_names10, model_names11]
    

img_sizes = [160, 320, 640, 800, 960, 1120]
img_batch_sizes = [1, 2, 5, 8, 10]

In [82]:
with open('yolo_predict_cpu.csv', 'w', newline='') as file:   # file writer
    writer = csv.writer(file)
    field = ['cpu_name', 
             'gpu_name', 
             'gpu_memory',
             'used_gpu',
             'ram_memory', 
             'model_name', 
             'model_param_dict', 
             'num_all_params',
             'model_img_size', 
             'image_batch_size',
             'predicting_times'
            ]
    writer.writerow(field)
    
    # определим константы
    cpu_name = cpu
    gpu_name = gpu
    gpu_memory = gpu_mem
    used_gpu = used_gpu
    ram_memory = ram

    # варьируемые переменные
    for model_group in models:
        for a in model_group:
            model_name = a
            model = YOLO(model_name)
            model_param_dict, num_all_params = count_parameters(model)
            for k in img_batch_sizes:
                image_batch_size = k
                for j in img_sizes:
                    model_img_size = j
                    random_images = get_random_images_from_folder(folder_path, image_batch_size)
                    since = time.time()
                    model.predict(
                        source=random_images,                          
                        imgsz=model_img_size,            
                        device='cpu',
                        save=False
                    )
                    predicting_times = time.time() - since
                    row = [cpu_name, 
                           gpu_name, 
                           gpu_memory, 
                           used_gpu, 
                           ram_memory, 
                           model_name,
                           model_param_dict,
                           num_all_params,
                           model_img_size, 
                           image_batch_size,
                           predicting_times]
                    # torch.cuda.empty_cache()
                    writer.writerow(row)


0: 128x160 3 cars, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 160)

0: 256x320 1 person, 1 car, 35.0ms
Speed: 1.0ms preprocess, 35.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 512x640 3 cars, 1 airplane, 1 bus, 65.0ms
Speed: 2.0ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 608x800 1 motorcycle, 1 airplane, 2 trucks, 83.0ms
Speed: 4.0ms preprocess, 83.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 736x960 2 persons, 7 cars, 1 truck, 101.0ms
Speed: 5.0ms preprocess, 101.0ms inference, 1.0ms postprocess per image at shape (1, 3, 736, 960)

0: 864x1120 7 cars, 1 bench, 126.0ms
Speed: 5.0ms preprocess, 126.0ms inference, 2.0ms postprocess per image at shape (1, 3, 864, 1120)

0: 160x160 (no detections), 19.0ms
1: 160x160 1 truck, 19.0ms
Speed: 0.5ms preprocess, 19.0ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 160)

0: 3